## This script makes the FLARE input files and configuration files for sailor output results (after filtering), using default parameters

In [1]:
from glob import glob
import os
import json
import yaml
import subprocess

cwd = os.getcwd()
main_directory = cwd

print(main_directory)

/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks


In [2]:
input_dir = main_directory + '/sailor_output/filtered_sailor_bed'
output_dir = main_directory + '/flare_filtered'
jsons_dir = main_directory + '/flare_filtered/flare_inputs/'
fasta = '/tscc/projects/ps-yeolab3/bay001/annotations/hg38/hg38.fa'
regions = '/tscc/projects/ps-yeolab3/ekofman/ReferenceData/peakcalling_regions/hg38/hg38_exons_and_introns_new'
edit_type = 'ct'
fdr_threshold = 0.1
max_merge_dist = 15
cluster_config = '/tscc/nfs/home/q2liang/FLARE_profile'
snakemake_dir_path = '/tscc/projects/ps-yeolab4/software/stamp/0.99.0/bin/stamp/'

subprocess.run(["mkdir", output_dir])
subprocess.run(["mkdir", jsons_dir])

CompletedProcess(args=['mkdir', '/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/flare_filtered/flare_inputs/'], returncode=1)

In [3]:
samples = sorted(glob(os.path.join(input_dir, '*.bed')))
samples = [i.split('/')[-1].split('.0.5')[0] for i in samples] 

In [4]:
og_sailor_output_dir = '/tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/FM_fix_rbfox2_tdp43_HEK_cells/sailor_output'
stamp_sites = sorted(glob(os.path.join(input_dir, '*.bed')))
fwd_bw = sorted(glob(os.path.join(og_sailor_output_dir, '8_bw_and_bam/*.fwd.sorted.bw')))
rev_bw = sorted(glob(os.path.join(og_sailor_output_dir, '8_bw_and_bam/*.rev.sorted.bw')))
bam = sorted(glob(os.path.join(og_sailor_output_dir, '8_bw_and_bam/*.bam_filtered_merged.sorted.bam')))

commands = []

for s in samples:
    sample_dict = {}
    sample_dict['label'] = s
    sample_dict['output_folder'] = output_dir
    sample_dict['stamp_sites_file'] = [i for i in stamp_sites if s in i][0]
    sample_dict['forward_bw'] = [i for i in fwd_bw if s in i][0]
    sample_dict['reverse_bw'] = [i for i in rev_bw if s in i][0]
    sample_dict['fasta'] = fasta
    sample_dict['regions'] = regions
    sample_dict['edit_type'] = edit_type
    sample_dict['fdr_threshold'] = fdr_threshold
    sample_dict['max_merge_dist'] = max_merge_dist
    sample_dict['bam'] = [i for i in bam if s in i][0]
    sample_dict['snakemake_dir_path'] = snakemake_dir_path
    
    with open(os.path.join(jsons_dir, '{}_flare_inputs_filtered.json'.format(s)), 'w') as file:
        json.dump(sample_dict, file)
        
    command = 'module load snakemake; snakemake '
    command += '--profile {} '.format(cluster_config)
    command += '--configfile {} '.format(os.path.join(jsons_dir, '{}_flare_inputs_filtered.json'.format(s)))
    command += ' -j 30 '
    commands.append(command)
print(len(commands))
print(commands[0])

12
module load snakemake; snakemake --profile /tscc/nfs/home/q2liang/FLARE_profile --configfile /tscc/lustre/ddn/scratch/q2liang/isSTAMP_publication_scripts/example_analysis_notebooks/flare_filtered/flare_inputs/isSTAMP10_01_flare_inputs_filtered.json  -j 30 


In [5]:
with open('flare_filtered.qsub', 'w') as file:
    file.write('#!/bin/bash\n#PBS -V\n#PBS -l walltime=48:00:00\n#PBS -l nodes=1:ppn=1\n#PBS -q home-yeo\n#PBS -t 1-'+str(len(commands))+'\ncd $PBS_O_WORKDIR\n\n')
    for i in range(len(commands)):
        file.write('cmd[{0}]="{1}"\n'.format(i+1, commands[i]))
    file.write('eval ${cmd[$PBS_ARRAYID]}')